<a href="https://colab.research.google.com/github/women-in-ai-ireland/June-2024-Group-002/blob/main/Study_Pal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, message="`do_sample` is set to `False`.")

In [2]:
# Install required libraries
!pip install pytorch torchvision torchaudio
!pip install transformers==4.30
!pip install langchain sentence_transformers huggingface-hub
!pip install pypdf
!pip install -U langchain-community
!pip install bitsandbytes
!pip install faiss-cpu langchain-openai tiktoken unstructured selenium newspaper3k textstat
!pip install accelerate

!pip install langchain-huggingface
!pip install sentence-transformers==2.2.2
!pip install InstructorEmbedding

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pytorch)
  Using cached transformers-4.30.0-py3-none-any.whl.metadata (113 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.30.0-py3-none-any.whl (7.2 MB)
Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfu

In [3]:
from google.colab import drive, userdata
import os
import pickle
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [4]:
# Mount Google Drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/MyDrive/WAI_project2/"

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Set HF token
hf_token = userdata.get('HF_TOKEN')

Mounted at /content/gdrive
Using device: cuda


In [5]:
# Function to initialize Hugging Face Instruct Embeddings
def initialize_huggingface_embeddings(model_name="hkunlp/instructor-xl", device="cuda"):
    """
    Initializes Hugging Face Instruct Embeddings model.

    Args:
    - model_name (str): Name of the Hugging Face model.
    - device (str): Device to run the model on.

    Returns:
    - embeddings: Initialized Hugging Face Instruct Embeddings model.
    """
    return HuggingFaceInstructEmbeddings(model_name=model_name, model_kwargs={"device": device})
instructor_embeddings = initialize_huggingface_embeddings()


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [6]:
# set local path to store embeddings ***replace with SingleStore, AWS or similar
embedding_store_path = f"{root_dir}/embedding_store"

In [17]:
# defines the parameters to use the recursive text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 10,
    length_function = len,
)

In [18]:
def ingest_and_chunk_pdfs():
    loader = DirectoryLoader(f"{root_dir}", glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    texts = text_splitter.split_documents(documents)
    return texts

In [19]:
# Function to store embeddings
def store_embeddings(docs, embeddings, store_name, path):
    """
    Stores embeddings in FAISS format and saves to a pickle file.

    Args:
    - docs (list): List of documents.
    - embeddings: Embedding model.
    - store_name (str): Name of the embedding store.
    - path (str): Path to the directory where embeddings will be stored.
    """
    vector_store = FAISS.from_documents(docs, embeddings)
    with open(os.path.join(path, f"faiss_{store_name}.pkl"), "wb") as f:
        pickle.dump(vector_store, f)

In [20]:
# Function to load embeddings
def load_embeddings(store_name, path):
    """
    Loads embeddings from a pickle file.

    Args:
    - store_name (str): Name of the embedding store.
    - path (str): Path to the directory where embeddings are stored.

    Returns:
    - vector_store: Loaded FAISS vector store.
    """
    with open(os.path.join(path, f"faiss_{store_name}.pkl"), "rb") as f:
        vector_store = pickle.load(f)
    return vector_store


In [21]:
def setup_embeddings():
    torch.cuda.empty_cache()

    # Read and chunk data
    texts = ingest_and_chunk_pdfs()

    # Create and store embeddings
    vector_store = FAISS.from_documents(texts, instructor_embeddings)
    with open(os.path.join(embedding_store_path, f"faiss_instructEmbeddings.pkl"), "wb") as f:
        pickle.dump(vector_store, f)

    # Return texts and vector_store for verification
    return texts, vector_store

# Call the setup function once to create and store embeddings
texts, vector_store = setup_embeddings()

In [22]:
def initialize_model_and_tokenizer():
    quantization_config = BitsAndBytesConfig(load_in_4bit=True)
    model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", quantization_config=quantization_config)
    tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b", quantization_config=quantization_config, model_max_length=256)
    return model, tokenizer


def retrieve_relevant_chunks(question, vector_store, num_chunks=1):
    docs = vector_store.similarity_search(question, k=num_chunks)
    return docs

In [23]:
def format_prompt(question, chunks):
    context = "\n".join([chunk.page_content for chunk in chunks])
    prompt = f"Provide an answer to the following question using only the context provided: {question}? " \
             f"If you cannot answer this question from the information provided, respond with 'There is insufficient information to answer this question.'\n\n{context}"
    return prompt

def gen_answer(prompt, tokenizer, model, max_length=200, temperature=0.7, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        response = model.generate(
            inputs,
            max_new_tokens=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )
    answer = tokenizer.decode(response[0], skip_special_tokens=True)
    return answer.strip()

In [24]:

# Main function to process the question and generate the answer
def main(question):
    torch.cuda.empty_cache()
    vector_store = load_embeddings(store_name='instructEmbeddings', path=embedding_store_path)
    model, tokenizer = initialize_model_and_tokenizer()
    torch.cuda.empty_cache()
    relevant_chunks = retrieve_relevant_chunks(question, vector_store)
    torch.cuda.empty_cache()
    prompt = format_prompt(question, relevant_chunks)
    answer = gen_answer(prompt, tokenizer, model)
    torch.cuda.empty_cache()
    return answer

In [30]:
# Example usage
question = input("Enter your question: ")
answer = main(question)
print(answer)

Enter your question: How does the Hadley Cell work?


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Provide an answer to the following question using only the context provided: How does the Hadley Cell work?? If you cannot answer this question from the information provided, respond with 'There is insufficient information to answer this question.'

The main cell to know is the Hadley Cell  
Here’s how it works: 

1. A number of satellites, which have been placed into polar orbit, look for and photograph the earth’s polar regions. 

2. These images are then compared and correlated, and the images are combined to create a single, highly accurate, digital image of the polar regions. 

3. This image is then processed to give an indication of the Earth’s temperature, and is used by the Hadley Centre to produce a digital, high-resolution, 12-hourly temperature record for the polar regions. 

4. This record is then used to make predictions and projections of future climate change.

5. The Hadley Centre is a Met Office global meteorology and climate research centre. It is also a major compone